In [31]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import data_analysis_tools as tools
import re
import random

## BASELINE

In [32]:
OVERRIDE_OLD_FIXED = False
TEST = False

In [33]:
base_path = f"{os.getcwd()}/../logs_good/"
baseline_paths = [
	f"{base_path}logs_publicos/battery_1/",

	# f"{base_path}tests/",

	# f"{base_path}baseline/4_ghosts_4_target_spread/",
	# f"{base_path}baseline/2_ghosts_0_target_spread/",
	# f"{base_path}baseline/2_ghosts_fixed_target_2_-5/",
	# f"{base_path}baseline/battery_1_2_ghosts_fixed_target_4_-5/",
]

neuroevolution_paths = [
	f"{base_path}logs_publicos/battery_2/B1/",
	f"{base_path}logs_publicos/battery_2/B1-path-grading-hack/",
	f"{base_path}logs_publicos/battery_2/B1-path-grading-simple/",
	f"{base_path}logs_publicos/battery_2/B2/",
	f"{base_path}logs_publicos/battery_2/B3/",
	f"{base_path}logs_publicos/battery_2/NB4/",
	f"{base_path}logs_publicos/battery_2/NB4-path-grading/",

	# f"{base_path}neuroevolution/battery_3_0/",
]

neat_paths = [
	# f"{base_path}NEAT/battery_3_0/",
	# f"{base_path}NEAT/saved_logs/no_speciation/",

	# f"{base_path}NEAT/history_size_100/B1/",
	# f"{base_path}NEAT/history_size_100/B1-path-grading-hack/",
	# f"{base_path}NEAT/history_size_100/B1-path-grading-simple/",
	# f"{base_path}NEAT/history_size_100/B2/",
	# f"{base_path}NEAT/history_size_100/B3/",
	# f"{base_path}NEAT/history_size_100/NB4/",
	# f"{base_path}NEAT/history_size_100/NB4-path-grading/",

	# f"{base_path}NEAT/saved_logs/speciated/B1/",
	# f"{base_path}NEAT/saved_logs/speciated/B1-path-grading-hack/",
	# f"{base_path}NEAT/saved_logs/speciated/B1-path-grading-simple/",
	# f"{base_path}NEAT/saved_logs/speciated/B2/",
	# f"{base_path}NEAT/saved_logs/speciated/B3/",
	# f"{base_path}NEAT/saved_logs/speciated/NB4/",
	# f"{base_path}NEAT/saved_logs/speciated/NB4-path-grading/",
]
selected_paths = baseline_paths + neuroevolution_paths + neat_paths

In [34]:
def prepare_data_fixed(path, selected_runs):
	for selected_run in selected_runs:
		this_path = f"{path}{selected_run}.data"
		if not os.path.exists(this_path + "_fixed") or OVERRIDE_OLD_FIXED:
			if not os.path.exists(this_path) or OVERRIDE_OLD_FIXED:
				#join data if necessary
				tools.join_data(this_path)
			tools.quote_gene(this_path)

def get_availiable_runs(filenames_list):
	selected_runs_set = set()
	for filename in filenames_list:
		found = re.findall("[0-9]+", filename)
		if len(found) > 0:
			selected_runs_set.add(found[0])
	return list(selected_runs_set)

def analyse_all_in_path(path):
	in_path_df_dict = dict()
	selected_runs = get_availiable_runs(os.listdir(path))
	prepare_data_fixed(path, selected_runs)
	errors = list()

	for run in selected_runs:
		this_baseline_mode = None
		this_ann_layers = None
		print(path, run)
		run_dict = dict()
		run_dict["run_id"] = run
		run_dict["path"] = path
		with open(f"{path}{run}.conf", 'r') as f:
			for line in f.readlines():
				if tools.ANN_MODE_STRING in line:
					this_mode = line.replace(tools.ANN_MODE_STRING, "")
					run_dict["mode"] = this_mode
				elif tools.ANN_LAYERS_STRING in line:
					this_layers = line.replace(tools.ANN_LAYERS_STRING, "")
					run_dict["layers"] = this_layers

		run_dict["df"], load_errors = tools.load_dataframe(path, run)
		in_path_df_dict[run] = run_dict
		errors.extend(load_errors)
		if TEST == True:
			return in_path_df_dict, errors

	return in_path_df_dict, errors


## Do it

In [35]:
errors = list()
paths_dict = dict()
for path in selected_paths:
	paths_dict[path], load_errors = analyse_all_in_path(path)
	errors.extend(load_errors)
	if TEST == True:
		break

# run count
run_count = 0
for index, path_dict in paths_dict.items():
	for _, run_dict in path_dict.items():
		run_count = run_count + 1
		errors.extend(tools.generate_run_report_from_dict(run_dict))

print("Path count: ", len(paths_dict.keys()))
print("Run count: ", run_count)
print("Errors:\n" + str(errors))

FileNotFoundError: [Errno 2] No such file or directory: 'C:/extincao-qpd/logs_good/tests/1675110905.data'